# Additional parsing of the dataset

Here, we created the script where we additionally parsed data and combined training13b.json and retrived_articles_sampled.json file
We removed data which may not be relevant for eaxtraction

In [ ]:
import json


# retreived_articles_sampled.json path
retreived_articles_sampled_path = "../../dataset/retrieved_articles_sampled.json"

# retreived_articles_sampled.json path
training13b_path = "../../dataset/training13b.json"

# link to parsed json file
parsed_data_path = "../../dataset/parsed_data_final.json"

In [33]:
# Load sample document dataset
with open(retreived_articles_sampled_path, 'r', encoding='utf-8') as f:
    retreived_articles_sampled_data = json.load(f)['results']  # assumes your sample is under the 'results' key

# Load BioASQ training dataset
with open(training13b_path, 'r', encoding='utf-8') as f:
    training13b_data = json.load(f)['questions']  

In [34]:
# this is function that converts the PubMed ID to a link, as same as showed in the training13b.json
def convert_pid_to_link(pid):
    """
    Convert PubMed ID to link.
    """
    link_prefix = "http://www.ncbi.nlm.nih.gov/pubmed/"
    return link_prefix + pid if pid and link_prefix not in pid else pid

Now, we will extract relevant info from each dataset
retreived_articles_sampled.json -> [question id, question, error_rate, ground truth list, articles (with info)]
training_articles_sampled.json -> [question id, ground truth snippets]

We will first create dictionary from training13b.json, because we need only groun truth snippets


In [35]:
training13b_gt_snippets = {}
for element in training13b_data:
    training13b_gt_snippets[element['id']] = element['snippets']

In [36]:
# extract the text from the retrieved articles
retreived_articles_sampled_relevant_info = []
for object in retreived_articles_sampled_data:
    # we need for each article to convert pid from the id to link
    relevant_articles = []
    for article in object['details']:
        article['pid'] = convert_pid_to_link(article['pid'])
        relevant_articles.append(article)
    
    relevant_element = {
        'qid': object['qid'],
        'question': object['question'],
        'ground_truth_documents_pid': [convert_pid_to_link(gt_pid) for gt_pid in object['ground_truth']], 
        'error_rate': object['error_rate'],
        'ground_truth_snippets': training13b_gt_snippets[object['qid']],
        'all_retreived_articles': relevant_articles
    }
    retreived_articles_sampled_relevant_info.append(relevant_element)

print(retreived_articles_sampled_relevant_info[0:3])

[{'qid': '55031181e9bde69634000014', 'question': 'Is Hirschsprung disease a mendelian or a multifactorial disorder?', 'ground_truth_documents_pid': ['http://www.ncbi.nlm.nih.gov/pubmed/6650562', 'http://www.ncbi.nlm.nih.gov/pubmed/15829955', 'http://www.ncbi.nlm.nih.gov/pubmed/12239580', 'http://www.ncbi.nlm.nih.gov/pubmed/20598273', 'http://www.ncbi.nlm.nih.gov/pubmed/23001136', 'http://www.ncbi.nlm.nih.gov/pubmed/8896569', 'http://www.ncbi.nlm.nih.gov/pubmed/15858239', 'http://www.ncbi.nlm.nih.gov/pubmed/15617541', 'http://www.ncbi.nlm.nih.gov/pubmed/21995290'], 'error_rate': {'value': 1.0, 'details': '9 found out of 9'}, 'ground_truth_snippets': [{'offsetInBeginSection': 131, 'offsetInEndSection': 358, 'text': 'Hirschsprung disease (HSCR) is a multifactorial, non-mendelian disorder in which rare high-penetrance coding sequence mutations in the receptor tyrosine kinase RET contribute to risk in combination with mutations at other genes', 'beginSection': 'abstract', 'document': 'http:

In [37]:
# write this info to the json file
parsed_data_final = {
    'data': retreived_articles_sampled_relevant_info
}

data_serialized = json.dumps(parsed_data_final, indent = 4)

with open(parsed_data_path, 'w') as outfile:
    outfile.write(data_serialized)


## TF-IDF retreival

In [ ]:
Because 